# Public Key Cryptography I

## 1 Shamir’s three-pass protocol

Alice and Bob want the implement Shamir’s three-pass protocol using the Vernam cipher, i.e. one-time pad. This is supposed to provide perfect secrecy. Is the following protocol secure?

![Vernam ciphers (otp) commute!](img/VernamCiphers.png)

__Your Task__: Can You compute the message? Make an example with $M = 010110111101$,
$K_A = 101101110100$, and $K_B = 001011011011$.

## 2 Diffie Hellman

_Alice_ and _Bob_ agree to use $n = 13$ and $e = 11$. Alice chooses her secret number $a = 5$, whereas Bob chooses $b = 7$.

__Your Task__: What are the requirements for $n$ and $e$? Are they fullfilled? Describe the key agreement protocol step by step using the above assumptions about a and b. What is the common secret key?

$e$ must be a generator for $\mathbb{Z}_n^*$, this is true:

In [1]:
Zp = []
for i in range(1,13):
    Zp.append(11**i%13)

Zp.sort()
print(Zp)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [2]:
print(11**5%13)
print(11**7%13)
print(11**(5*7)%13)

7
2
6


Alice sends $e^a \mod n$ to Bob

$$ 11^5 \mod 13 = 7 $$

Bob sends $e^b \mod n$ to Alice
$$ 11^7 \mod 13 = 2 $$

Both of them compute the secret key k:
$$ 7^7 \mod 13 = 2^5 \mod 13 = 11^{5\cdot 7} \mod 13 = 6 $$

## 3 Discrete Logarithm Problem

Assume Mallory intercepts the message $A = 9$ from Alice to Bob and B = 3 from Bob to Alice.
He also knows $n = 13$ and $g = 11$.

__Your Task__: Suppose Mallory wants to know the common key. Describe his steps to find this key!

## 4 Attack on textbook RSA

The public key $(n,e) = (2537,13)$ was used to encrypt the plaintext $M$. Eve intercepts the ciphertext $C = 2081$.

__Your Task__: Show how Eve computes the plaintext $M$!

## 5 Attack on textbook RSA — small exponent e

Frequently, the exponent $e$ in the public key $(n,e)$ is choosen very small, say $e = 3$. Hence, encryption of $m$ is very fast

$$ c = m^3 \mod n$$

because modular exponentiation with small exponent is fast.

Unfortunately, this is is bad, if a small message, $m < n^{\frac{1}{3}}$ is encrypted, because there is no modular reduction and the attacker only has to compute the cubic root of $c$.

In the preceding exercise we constructed an attack which works for arbitrary messages $m, (1 < m < n−2)$. To this end, we assume $e = 3$ and send the same message to three people with public keys $(n_1 ,e), (n_2 ,e)$, and $(n_3 ,e)$:

$$ c_1 = m^3 \mod n_1,\ c_2 = m^3 \mod n_2,\ c_3 = m^3 \mod n_3 .$$

Furthermore we assume, that the moduli $n_1, n_2$, and $n_3$ are pairwise co-prime, i.e. $\gcd(n_i ,n_j)=1,\ 
for\ i \neq j$.

According to the chinese remainder theorem (CRT), there is a solution to these three linear congruences

$$ c_1 = m^3 \mod n_1,\ c_2 = m^3 \mod n_2,\ c_3 = m^3 \mod n_3 .$$

First let $n = n_1 \cdot n_2 \cdot n_3$ and

$$ N_1 =\frac{n}{n_1}=n_2n_3,\ \ N_2 =\frac{n}{n_2}=n_1n_3, \ \  N_3 =\frac{n}{n_3}=n_1n_2$$

Because $n_i$ and $n_j$ are co-prime if $i \neq j$, it follows that $\gcd(n_i,N_i) = 1$. Consequently, we can compute the (multiplicative) inverse $y_i$ of $N_i$ modulo $n_i$ such that

$$ N_1 y_1 \equiv 1\ (\text{mod}\ n_1 ),\ N_2 y_2 \equiv 1\ (\text{mod}\ n_2 ),\ N_3 y_3 \equiv 1\ (\text{mod}\ n_3 ) .$$

Then the simultaneous solution of the system of linear congruences is

$$ m^3 =\sum_{i=1}^{3}c_i N_i y_i = c_1 N_1 y_1 +c_2 N_2 y_2 + c_3 N_3 y_3 .$$

Here $m^3$ is unique up to a multiple of $n_1 n_2 n_3$. Because $m^3$ is typically smaller than $n_1 n_2 n_3$ we can just take the cube root of $m^3$ to find $m$.


__Your Task__: Assume the message $m$ is sent to 3 different people using textbook RSA, with moduli $n_1 = 377,\ n_2 = 391$, and $n_3 = 589$. You get hold of the corresponding ciphertexts

$$330 = m^3 \mod 377$$
$$34 = m^3 \mod 391$$
$$419 = m^3 \mod 589$$

Compute $m =\sqrt[3]{x}$ using the CRT, where $x = m^3$ satisfies the system of linear congruences

$$ x \equiv 330 \mod 377 $$
$$ x \equiv 34  \mod 391 $$
$$ x \equiv 419 \mod 589 $$

Use python in a Jupyter notebook. Use the (extended) Euklidean algorithm to compute the inverses and find or invent a python code, which implements the CRT.

In [37]:
def extended_euclidian_algorithm(a, b):
    if (a>0 and b>0):
        x,y = 0, 0
        gcd = 0
        s = [1, 0]
        t = [0, 1]
        q = [0,0]
        if (b>a):
            r = [b,a]
        else:
            r = [a,b]
        i = 1
        while (r[i] != 0):
            # Get the quotient and remainder
            q_i, r_i = divmod(r[i-1], r[i])
            q.append(q_i)
            r.append(r_i)
            s.append(s[i-1] - q[i+1] * s[i])
            t.append(t[i-1] - q[i+1] * t[i])
            i = i+1
        if (b>a):
            gcd, x, y = r[i-1], t[i-1], s[i-1]
        else:
            gcd, x, y = r[i-1], s[i-1], t[i-1]
    else:
        gcd, x, y = 0,0,0
    return gcd,x,y

def chinese_remainder_theorem(remainder_1, modulo_1, remainder_2, modulo_2, remainder_3, modulo_3):
    m = modulo_1 * modulo_2 * modulo_3
    M_1 = m/modulo_1
    M_2 = m/modulo_2
    M_3 = m/modulo_3
    _,_,d_1 = extended_euclidian_algorithm(modulo_1, M_1)
    if (d_1<0):
        d_1 = d_1 + modulo_1
    _,_,d_2 = extended_euclidian_algorithm(modulo_2, M_2)
    if (d_2<0):
        d_2 = d_2 + modulo_2
    _,_,d_3 = extended_euclidian_algorithm(modulo_3, M_3)
    if (d_3<0):
        d_3 = d_3 + modulo_3
    return remainder_1*M_1*d_1 + remainder_2*M_2*d_2 + remainder_3*M_3*d_3

m3 = chinese_remainder_theorem(330, 377, 34, 391, 419, 589)
print(m3 % 377)
print(m3 % 391)
print(m3 % 589)
print(m3 ** (1. /3))

330.0
34.0
419.0
3924.1905678776743


## 6 Attack on textbook RSA — common module n

Suppose the CTO of a company wants that all employees use the same module $n$. The individual employees have pairwise different $(e_i , d_i )$. Suppose, two employees $A$ and $B$ have the public keys $(n,e_A)$ and $(n,e_B)$ where $\gcd(e_A,e_B) = 1$.

Now the administration sends the encrypted message $m$ to the two employees

$$c_A = m^{e_A} \mod n $$
$$ c_B = m^{e_B} \mod n$$

We will now show, that Eve is able to compute $m$ if she knows the two ciphertexts $c_A$ and $c_B$. She first computes $a$ and $b$ such that

$$ae_A +be_B = 1$$

She does it using the extended Euclidean algorithm which works because $\gcd(e_A,e_B)=1$. Then she computes

$$ c_A^{a}c_B^{b} \equiv (m^{e_A})^a (m^{e_B})^b \equiv m^{ae_A +be_B} \equiv m^1 \equiv m$$

Hence, as promised, she can compute $m$.

__Your Task__: Design a example with small numbers which demonstrates, this attack! Assume $n = 11\cdot 13$, i.e. $p = 11$ and $q = 13$.

## 7 Elgamal

The prime number $p=13$ and the generator $g=3$ were used. Check if 3 is a generator: otherwise use the next larger number after 3. Bob chooses the secret key $sk_B = j = 3$ and Alice $sk_A = i = 4$.

__Your Task__: Compute all intermediate results if Alice wants to securely send the message $m = 12$ to Bob.

## 8 Elgamal 2nd

Alice uses the private key $a=1751$ and computes the public key $(p=2357,\alpha=2,\alpha^a =1185)$.
Now Bob wants to encrypt the message $m = 2035$. He uses the random $k = 1520$.

__Your Task__: Compute the encrypted message and show how Alice decrypts the message.

#### Alice

Primzahl $p$

Erzeugende $a \in \mathbb{Z}_p^*$

$s_{K_b} = j\in {2,...,p-2}$

$p_{K_b} = \beta = a^j \mod p $

$(\beta, p, a) \to$ Bob

#### Bob
$s_{K_a} = p_{K_a} = a^i mod p$

Temporärer Schlüssel $K_E = p_{K_a} = a^i\mod p$

Session Key $K_M = \beta^i \mod p$

Ciphertext $ y = x\cdot K_M \mod p$

$(y, K_E) \to$ Alice

#### Alice

$K_M = K_E^j \mod p$

$x = K_M^{-1}y \mod p$

In [42]:
# Primzahl
p = 2357
# Erzeugende
alpha = 2
# Private Key Alice
a = 1751
# Public Key Alice
beta = alpha ** a % p
print("Public Key Beta: " + str(beta))
# Bob's Private Key
k = 1520
# Temporärer Schlüssel
K_E = alpha ** k % p
print("Temp. Key: " + str(K_E))
# Session key
K_M_1 = beta ** k % p
print("Session Key von Bob: " + str(K_M_1))
# Klartext
m = 2035
# Ciphertext
y = m * K_M % p
print("Ciphertext: " + str(y))
# Alice berechnet Session Key
K_M_2 = K_E ** a % p
print("Session Key von Alice: " + str(K_M_2))
# Alice berechnet Klartext
x = pow(K_M_2, p-2, p) * y % p
print("Klartext Alice: " + str(x))

Public Key Beta: 1185
Temp. Key: 1430
Session Key von Bob: 2084
Ciphertext: 697
Session Key von Alice: 2084
Klartext Alice: 2035
